# What can I tell about start and end tokens

In [1]:
%cd ..

C:\Users\Stefan.Beuchert\Desktop\backup_from_kubectl


In [2]:
# config
number_of_features = 10 # the default value
number_of_samples = 5000 # the default value

source_path_for_data = f'data/Data_Preparation/lime_ns_{number_of_samples}_nf_{number_of_features}_after_frequency_analysis.json'

In [3]:
from src.get_data import enhance_data_with_question_type

import pandas as pd
import numpy as np
import spacy

## Get Data

In [4]:
# load data
data_df = pd.read_json(source_path_for_data)
# data_df['explanation'] = [pd.DataFrame(exp) for exp in data_df['explanation'].tolist()]
data_df = enhance_data_with_question_type(data_df)
data_df.head()

,question,answers,context,document_id,is_impossible,usage,prediction,explanation,question_type
37495,Wie viele Amerikaner fühlen sich mehr als eine...,"[{'answer_id': 37854, 'document_id': 41057, 'q...",Race__United_States_Census_\n\n=== „Race“ ===\...,41057,False,test,fast sieben Millionen,"{'position_id': {'0': 0, '1': 1, '2': 2, '3': ...",wie viel
36553,Wer wanderte nach North Carolina Anfang des 20...,"[{'answer_id': 36895, 'document_id': 40789, 'q...",North_Carolina\n\n==== Europäische Amerikaner ...,40789,False,test,eine Gruppe orthodoxer Einwanderer aus der Ukr...,"{'position_id': {'0': 0, '1': 1, '2': 2, '3': ...",wer
36597,Was gehört zum Stadtkirchenverband Hannover?,"[{'answer_id': 36938, 'document_id': 40797, 'q...",Hannover\n\n==== Evangelisch-lutherische Kirch...,40797,False,test,Alle landeskirchlichen evangelischen Kirchenge...,"{'position_id': {'0': 0, '1': 1, '2': 2, '3': ...",was
37392,Wann starb Athanasius der Große?,"[{'answer_id': 37749, 'document_id': 41075, 'q...","Athanasius_der_Große\nGriechische Ikone, Titul...",41075,False,test,2. Mai 373,"{'position_id': {'0': 0, '1': 1, '2': 2, '3': ...",wann
37351,Welche Länder liegen östlich des Irans?,"[{'answer_id': 37706, 'document_id': 41082, 'q...",Iran\n\n=== Antike und Mittelalter ===\nDas he...,41082,False,test,Indien und China,"{'position_id': {'0': 0, '1': 1, '2': 2, '3': ...",welche


## Extract start and end tokens as well as the corresponding POS tag

In [5]:
# get prediction tokens
prediction_list = data_df['prediction'].tolist()
prediction_list_split = [p.split() for p in prediction_list]

prediction_list_split_nan = []
for p in prediction_list_split:
    if len(p) == 0:
        prediction_list_split_nan.append([None])
    else:
        prediction_list_split_nan.append(p)

# get prediction pos
nlp = spacy.load("de_core_news_sm")

pos_prediction_list = []

for p in prediction_list_split_nan:
    if p[0] is not None:
        nlp_tags = nlp(' '.join(p))
        list_pos_tags = [i.pos_ for i in nlp_tags]
        
        pos_prediction_list.append(list_pos_tags)
        
    else:
        pos_prediction_list.append(p)

data_df['start_token'] = [p[0] for p in prediction_list_split_nan]
data_df['end_token'] = [p[-1] for p in prediction_list_split_nan]

data_df['start_token_pos'] = [p[0] for p in pos_prediction_list]
data_df['end_token_pos'] = [p[-1] for p in pos_prediction_list]

data_df['start_token'].fillna(value=np.nan, inplace=True)
data_df['end_token'].fillna(value=np.nan, inplace=True)
data_df['start_token_pos'].fillna(value=np.nan, inplace=True)
data_df['end_token_pos'].fillna(value=np.nan, inplace=True)

In [6]:
data_df['start_token']

37495          fast
36553          eine
36597          Alle
37392            2.
37351        Indien
            ...    
41111           Nur
38148       Cassava
37012            Im
38199    Allerdings
38841           NaN
Name: start_token, Length: 2176, dtype: object

## Get insights

In [7]:
#data_df[data_df['start_token'].notna()]['start_token'].value_counts().plot.bar()

In [8]:
#data_df['end_token'].value_counts().plot.bar()

In [9]:
#data_df['start_token_pos'].value_counts().plot.bar()

In [10]:
a = data_df['end_token_pos'].value_counts()#.plot.bar()

In [11]:
print(a)

NOUN     703
PROPN    624
NUM      259
PUNCT    220
VERB     114
AUX       58
X         34
ADV       31
ADJ       11
ADP        2
PRON       2
DET        1
PART       1
CCONJ      1
Name: end_token_pos, dtype: int64


In [12]:
# a.plot()

In [13]:
#a.plot.bar()

In [14]:
data_df['start_token_pos'].value_counts()#.plot.bar()

ADP      378
DET      360
NOUN     285
PROPN    249
NUM      216
ADV      174
X        165
ADJ       82
SCONJ     46
PUNCT     41
PRON      32
VERB      21
CCONJ      5
AUX        5
PART       2
Name: start_token_pos, dtype: int64

In [ ]:
# create am function for my explainer object that takes data_df (for each quesiton type) 
# and returns the top pos tokens for start and end with probabilities

In [53]:
def get_pos_preferences_for_start_and_end_tokens(data_df):
    
    # get prediction tokens
    prediction_list = data_df['prediction'].tolist()
    prediction_list_split = [p.split() for p in prediction_list]

    prediction_list_split_nan = []
    for p in prediction_list_split:
        if len(p) == 0:
            prediction_list_split_nan.append([None])
        else:
            prediction_list_split_nan.append(p)

    # get prediction pos
    nlp = spacy.load("de_core_news_sm")

    pos_prediction_list = []

    for p in prediction_list_split_nan:
        if p[0] is not None:
            nlp_tags = nlp(' '.join(p))
            list_pos_tags = [i.pos_ for i in nlp_tags]

            pos_prediction_list.append(list_pos_tags)

        else:
            pos_prediction_list.append(p)

    data_df['start_token'] = [p[0] for p in prediction_list_split_nan]
    data_df['end_token'] = [p[-1] for p in prediction_list_split_nan]

    data_df['start_token_pos'] = [p[0] for p in pos_prediction_list]
    data_df['end_token_pos'] = [p[-1] for p in pos_prediction_list]

    data_df['start_token'].fillna(value=np.nan, inplace=True)
    data_df['end_token'].fillna(value=np.nan, inplace=True)
    data_df['start_token_pos'].fillna(value=np.nan, inplace=True)
    data_df['end_token_pos'].fillna(value=np.nan, inplace=True)
    
    start_token_pos_df = data_df['start_token_pos'].value_counts().to_frame()
    start_token_pos_df['percent'] = (start_token_pos_df['start_token_pos'] / start_token_pos_df['start_token_pos'].sum()) *100
    
    end_token_pos_df = data_df['end_token_pos'].value_counts().to_frame()
    end_token_pos_df['percent'] = (end_token_pos_df['end_token_pos'] / end_token_pos_df['end_token_pos'].sum()) *100
    
    start_token_pos_df = start_token_pos_df.sort_values(by=['percent'], ascending=False)
    end_token_pos_df = end_token_pos_df.sort_values(by=['percent'], ascending=False)
    
    print(start_token_pos_df)
    print(end_token_pos_df)
    
    res = {
        'top_pos_for_start_token' : start_token_pos_df.index[0],
        'second_top_pos_for_start_token' : start_token_pos_df.iloc[0]['percent'],
        
        'top_pos_for_start_token_probability' : start_token_pos_df.index[1],
        'second_top_pos_for_start_token_probability' : start_token_pos_df.iloc[1]['percent'],
        
        
        'top_pos_for_end_token' : end_token_pos_df.index[0],
        'top_pos_for_end_token_probability' : end_token_pos_df.iloc[0]['percent'],
        
        'secondtop_pos_for_end_token' : end_token_pos_df.index[1],
        'second_top_pos_for_end_token_probability' : end_token_pos_df.iloc[1]['percent'],
    }
    
    print(res)

In [54]:
get_pos_preferences_for_start_and_end_tokens(data_df.head(100))

       start_token_pos    percent
PROPN               19  20.430108
DET                 14  15.053763
ADP                 13  13.978495
NOUN                11  11.827957
ADV                 10  10.752688
X                    7   7.526882
NUM                  7   7.526882
ADJ                  3   3.225806
PUNCT                3   3.225806
SCONJ                2   2.150538
VERB                 2   2.150538
CCONJ                1   1.075269
PRON                 1   1.075269
       end_token_pos    percent
NOUN              35  37.634409
PROPN             32  34.408602
NUM                8   8.602151
VERB               5   5.376344
ADV                4   4.301075
PUNCT              3   3.225806
AUX                3   3.225806
X                  2   2.150538
ADP                1   1.075269
{'top_pos_for_start_token': 'PROPN', 'second_top_pos_for_start_token': 20.43010752688172, 'top_pos_for_start_token_probability': 'DET', 'second_top_pos_for_start_token_probability': 15.053763440860216, 't

C:\Users\Stefan.Beuchert\AppData\Local\Temp\ipykernel_12100\87224739.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['start_token'] = [p[0] for p in prediction_list_split_nan]
C:\Users\Stefan.Beuchert\AppData\Local\Temp\ipykernel_12100\87224739.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['end_token'] = [p[-1] for p in prediction_list_split_nan]
C:\Users\Stefan.Beuchert\AppData\Local\Temp\ipykernel_12100\87224739.py:32: SettingWithCopyWarning: 
A value is trying to be set on a c

{'top_pos_for_start_token': 'PROPN',
 'second_top_pos_for_start_token': 20.43010752688172,
 'top_pos_for_start_token_probability': 'DET',
 'second_top_pos_for_start_token_probability': 15.053763440860216,
 'top_pos_for_end_token': 'NOUN',
 'top_pos_for_end_token_probability': 37.634408602150536,
 'secondtop_pos_for_end_token': 'PROPN',
 'second_top_pos_for_end_token_probability': 34.40860215053764}